In [17]:
import numpy as np
from keras import layers as ly
from keras.models import Model
from keras.utils import layer_utils
from keras.utils import get_file
from keras.preprocessing import image
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.initializers import glorot_uniform
import scipy.misc
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow.compat.v1 as tf
K.set_learning_phase(1)
K.set_image_data_format('channels_last')

In [34]:
def identity_block(X, f, filters, stage, block):
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = ly.Conv2D(filters = F1,
               kernel_size = (1, 1),
               strides = (1,1),
               padding = 'valid',
               name = conv_name_base + '2a',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = ly.BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = ly.Activation('relu')(X)
    
    X = ly.Conv2D(filters = F2,
               kernel_size = (f, f),
               strides = (1, 1),
               padding = "same",
               name = conv_name_base + '2b',
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = ly.BatchNormalization(axis = 3, name = bn_name_base + "2b")(X)
    X = ly.Activation("relu")(X)

    X = ly.Conv2D(filters = F3,
               kernel_size = (1, 1),
               strides = (1, 1), 
               padding = "valid", 
               name=conv_name_base+"2c", 
               kernel_initializer = glorot_uniform(seed=0))(X)
    X = ly.BatchNormalization(axis=3, name=bn_name_base+"2c")(X)

    X = ly.Add()([X_shortcut, X])
    X = ly.Activation("relu")(X)
    
    return X

In [35]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

out = [0.        0.        1.3454674 2.0318177 0.        1.3246754]


In [62]:
def convolutional_block(X, f, filters, stage, block, s=2):
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"
    
    F1, F2, F3 = filters
    
    X_shortcut = X
    
    X = ly.Conv2D(filters = F1,
                kernel_size = (1, 1),
                strides = (s, s),
                name = conv_name_base+"2a",
                kernel_initializer = glorot_uniform(seed=0))(X)
    X = ly.BatchNormalization(axis=3, name=bn_name_base+"2a")(X)
    X = ly.Activation("relu")(X)
    
    X = ly.Conv2D(filters = F2,
                kernel_size = (f, f),
                strides = (1, 1),
                padding = "same",
                name = conv_name_base+"2b",
                kernel_initializer = glorot_uniform(seed=0))(X)
    X = ly.BatchNormalization(axis=3, name=bn_name_base+"2b")(X)
    
    X = ly.Conv2D(filters = F3,
                 kernel_size = (1, 1),
                 strides = (1, 1),
                 padding = "valid",
                 name = conv_name_base+"2c",
                 kernel_initializer = glorot_uniform(seed=0))(X)
    X = ly.BatchNormalization(axis=3, name=bn_name_base+'2c')(X)
    
    X_shortcut = ly.Conv2D(filters = F3, 
                kernel_size = (1, 1),
                strides = (s, s),
                name=conv_name_base+"1", 
                kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = ly.BatchNormalization(axis=3, name=bn_name_base+"1")(X_shortcut)
    
    X = ly.Add()([X_shortcut, X])
    X = ly.Activation("relu")(X)
    
    return X

In [63]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

out = [0.        0.        0.        0.8666411 0.        0.4765577]


In [64]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    X_input = ly.Input(input_shape)
    
    X = ly.ZeroPadding2D((3, 3))(X_input)
    
    # STAGE 1
    X = ly.Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = ly.BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = ly.Activation('relu')(X)
    X = ly.MaxPooling2D((3, 3), strides=(2, 2))(X)
    
    # STAGE 2
    X = convolutional_block(X, f = 3, s = 1,
                            filters = [64, 64, 256], 
                            stage = 2, 
                            block = "a")
    X = identity_block(X, 3, [64, 64, 256], stage = 2, block = "b")
    X = identity_block(X, 3, [64, 64, 256], stage = 2, block = "c")
    
    # STAGE 3
    X = convolutional_block(X, f = 3, s = 2, 
                            filters = [128, 128, 512],
                            stage = 3,
                            block = "a")
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')
    
    # STAGE 4
    X = convolutional_block(X, f = 3, s = 2, 
                            filters = [256, 256, 1024],
                            stage = 4,
                            block = "a")
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')    
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')   
    
    # STAGE 5
    X = convolutional_block(X, f = 3, s = 2, 
                            filters = [512, 512, 2048],
                            stage = 5,
                            block = "a")
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')
    
    X = ly.AveragePooling2D((2, 2), name="avg_pool")(X)
    
    X = ly.Flatten()(X)
    X = ly.Dense(classes,
                 activation = "softmax",
                 name = "fc" + str(classes),
                 kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X, name = "ResNet50")
    
    return model

In [65]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)

ValueError: The name "res2_branch2a" is used 2 times in the model. All layer names should be unique.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])